In [67]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

# NOTE IN WASEEM_TRAIN.csv indices are ordered by sample_key used in validation csv.

K = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
logits = np.loadtxt('../trained_models/waseem/logits_epoch.txt')
labels = pd.read_csv('../datasets/waseem/waseem_train.csv', header=0, sep='\t').label.values
print(labels)

[3 3 3 ... 3 3 3]


In [68]:
df = pd.read_csv('../trained_models/waseem/noiserank_val.tsv', sep='\t', header=None) # .label.values
df.head()
val_ids = df[0].values
val_vlabels = df[3].values
print(val_ids)
print(val_vlabels)

[ 818 4694 1357 ...  196 1251  767]
[1 1 1 ... 1 1 1]


In [93]:
from sklearn.metrics import f1_score, classification_report, precision_recall_fscore_support
import sys
import os
import pandas as pd
import numpy as np
import glob
from pprint import pprint

def report_metrics(targets, preds, classes):
    C = np.max(classes) + 1
    per_class_accuracies = np.zeros((C,))
    for c in range(C):
        ind = np.where(classes == c)[0]
        class_acc = f1_score(targets[ind], preds[ind], average='micro')
        per_class_accuracies[c] = class_acc
    
    print('P/R/F1 (noise)', precision_recall_fscore_support(targets, preds, pos_label=0, average='binary'))
    print('f1_metrics (macro/unweighted mean)', f1_score(targets, preds, average='macro'))
    acc_classes = np.mean(per_class_accuracies)
    print('avg accuracy over classes', acc_classes, 'AvgErrorRate', 1 - acc_classes)
    
    print()
    acc = f1_score(targets, preds, average='micro')
    print('f1_metrics (accuracy/micro)', acc, 'ErrorRate', 1 - acc)
    print('P/R/F1 (clean)', precision_recall_fscore_support(targets, preds, pos_label=1, average='binary'))
    print('f1_metrics (weighted mean of f1)', f1_score(targets, preds, average='weighted'))
    # cr = classification_report(targets, preds)
    

def noise_filtering(logits, labels, K, val_ids, val_vlabels):
    # rev_sorted = -np.sort(-logits, axis=1)
    # items = rev_sorted[:, :K]
    # print(items)
    nb_cls = len(np.unique(labels))
    
    for k in K:    
        print('\n\n-------------starting for k={}-----------------'.format(k))
        rev_ind_sorted = np.argsort(-logits, axis=1)
        classes = rev_ind_sorted[:, :k]
        print('classes', classes.shape)
        
        # evaluating on validation set:
        # binary = np.zeros((len(val_ids), nb_cls))
        val_classes = classes[val_ids, :]
        val_labels = labels[val_ids]
        val_preds = np.zeros((len(val_ids)))
        print('val_classes', val_classes.shape)
        for i in range(len(val_ids)):
            binary = np.zeros((nb_cls))
            # binary[i][val_classes[i]] = 1
            binary[val_classes[i]] = 1
            val_preds[i] = binary[val_labels[i]]  # if its in topK, then pred=1 (clean)
        
        # print(val_classes)
        print(binary)
        print(val_preds)
        report_metrics(val_vlabels, val_preds, val_labels)
        if k == nb_cls:
            break
        # break
    
noise_filtering(logits, labels, K, val_ids, val_vlabels)




-------------starting for k=1-----------------
classes (5527, 1)
val_classes (4697, 1)
[0. 0. 0. 1.]
[1. 1. 1. ... 1. 1. 1.]
P/R/F1 (noise) (0.2890855457227139, 0.14803625377643503, 0.1958041958041958, None)
f1_metrics (macro/unweighted mean) 0.5499454673766601
avg accuracy over classes 0.48693414488618797 AvgErrorRate 0.5130658551138121

f1_metrics (accuracy/micro) 0.8286140089418778 ErrorRate 0.17138599105812224
P/R/F1 (clean) (0.8705828361633777, 0.940272614622057, 0.9040867389491243, None)
f1_metrics (weighted mean of f1) 0.8042606704879911


-------------starting for k=2-----------------
classes (5527, 2)
val_classes (4697, 2)
[1. 0. 0. 1.]
[1. 1. 1. ... 1. 1. 1.]
P/R/F1 (noise) (0.20103092783505155, 0.05891238670694864, 0.0911214953271028, None)
f1_metrics (macro/unweighted mean) 0.4999997263470838
avg accuracy over classes 0.46457746641255326 AvgErrorRate 0.5354225335874467

f1_metrics (accuracy/micro) 0.8343623589525229 ErrorRate 0.16563764104747714
P/R/F1 (clean) (0.86164779